In [44]:
import pandas as pd
import requests
import pyarrow.feather as feather

In [45]:
articles_df = pd.read_feather('Data/dataframes/article_dataframe_embedding_distance.feather')
l_paths_df = pd.read_feather('Data/dataframes/length_filtered_finished_paths.feather')
t_paths_df = pd.read_feather('Data/dataframes/time_filtered_finished_paths.feather')

In [46]:
articles_df['embeddings'].shape

(4604,)

In [47]:
l_paths_df

,hashedIpAddress,timestamp,durationInSec,path,rating,finished,failure_reason,start_article,target_article,identifier,full_path,simplified_path,distance,full_path_length,simplified_path_length
0,2a08d9702d4b75a2,1359754387,169,Capitalism;England;Great_Britain;Europe;Africa...,NaN,True,None,Capitalism,Zulu,7672,"[Capitalism, England, Great_Britain, Europe, A...","[Capitalism, England, Great_Britain, Europe, A...",2.0,5,5
1,6b039e9953cf075e,1241103290,15,The_Simpsons;Christianity;Pope,1.0,True,None,The_Simpsons,Pope,37967,"[The_Simpsons, Christianity, Pope]","[The_Simpsons, Christianity, Pope]",2.0,2,2
2,06fa249851e60497,1248658467,38,England;Industrial_Revolution;Rail_transport;T...,NaN,True,None,England,Train,13119,"[England, Industrial_Revolution, Rail_transpor...","[England, Industrial_Revolution, Rail_transpor...",2.0,3,3
3,3d23bd006236fc37,1249503330,156,Super_Mario_64;Europe;Sweden;Alfred_Nobel;Pota...,3.0,True,None,Super_Mario_64,Gunpowder,36564,"[Super_Mario_64, Europe, Sweden, Alfred_Nobel,...","[Super_Mario_64, Europe, Sweden, Alfred_Nobel,...",2.0,5,5
4,50268bda76b436a2,1248911623,56,Algol;Earth;Tellurium;Chemical_element;Ununoct...,2.0,True,None,Algol,Uranium,1725,"[Algol, Earth, Tellurium, Chemical_element, Un...","[Algol, Earth, Tellurium, Chemical_element, Un...",2.0,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40516,0d57c8c57d75e2f5,1290077297,299,Batman;Superman;Television;Internet;World_Wide...,4.0,True,None,Batman,Wikisource,4701,"[Batman, Superman, Television, Internet, World...","[Batman, Superman, Television, Internet, World...",6.0,8,8
40517,473d6ac602c2b198,1318968881,110,Scheme_programming_language;Functional_program...,NaN,True,None,Scheme_programming_language,Effects_of_global_warming,33669,"[Scheme_programming_language, Functional_progr...","[Scheme_programming_language, Functional_progr...",6.0,6,6
40518,476eb082417269a7,1353019083,501,Batman;Wikipedia_Text_of_the_GNU_Free_Document...,NaN,True,None,Batman,Wikisource,4701,"[Batman, Wikipedia_Text_of_the_GNU_Free_Docume...","[Batman, Crime, Society, Technology, Computer,...",6.0,13,10
40519,1e68f2ea6fb133d3,1357686675,186,"Horatio_Nelson,_1st_Viscount_Nelson;England;St...",3.0,True,None,"Horatio_Nelson,_1st_Viscount_Nelson",Upsilon_Andromedae_d,18649,"[Horatio_Nelson,_1st_Viscount_Nelson, England,...","[Horatio_Nelson,_1st_Viscount_Nelson, England,...",6.0,6,6


In [48]:
articles_df.head(1)

,article,article_unrendered_unicode,category,linkSource,linkTarget,distances,plain_text,incoming_links,num_hyperlinks,num_characters,hyperlink_density,embeddings,average_outgoing_article_title_cosine_distance
0,Áedán_mac_Gabráin,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,[subject.History.British_History.British_Histo...,Áedán_mac_Gabráin,"[Bede, Columba, Dál_Riata, Great_Britain, Irel...","{'10th_century': 3.0, '11th_century': 3.0, '12...",#copyright\n\nÁedán mac Gabráin\n\n2007 Sch...,0,11,11612,0.000947,"[-0.1292391121387481, 0.0236219353973865, -0.0...",0.785123


In [ ]:
# 1. extract the simplified path and start article => new df of 2 columns 
# 2. a function that iterates over the simplified paths and extracts the corresponding vector embeddings from the articles_df
# 3. apply the function
# 4. compute the cosine similarty between all the articles and the target article (add a new column to the df (list of cosine similarities))
# 5. then iterate over the df to average the cosine similarities for each article

# what this can tell us? are the articles that get high scores in cosine similarity to the target on average the ones that have high scores?
# if so this might indicate theat these articles tend to appear in specific paths, or that the target articles favor certain topics...

In [50]:
# 1. extract the simplified path and start article => new df of 2 columns 

df = l_paths_df[['simplified_path']]

df


,simplified_path
0,"[Capitalism, England, Great_Britain, Europe, A..."
1,"[The_Simpsons, Christianity, Pope]"
2,"[England, Industrial_Revolution, Rail_transpor..."
3,"[Super_Mario_64, Europe, Sweden, Alfred_Nobel,..."
4,"[Algol, Earth, Tellurium, Chemical_element, Un..."
...,...
40516,"[Batman, Superman, Television, Internet, World..."
40517,"[Scheme_programming_language, Functional_progr..."
40518,"[Batman, Crime, Society, Technology, Computer,..."
40519,"[Horatio_Nelson,_1st_Viscount_Nelson, England,..."


In [57]:
# 2. a function that iterates over the simplified paths and extracts the corresponding vector embeddings from the articles_df

# Define the function to get embeddings
def get_embeddings(simplified_path, articles_df):
    articles_df.copy()
    return articles_df[articles_df['article'].isin(simplified_path)]['embeddings'].tolist()

# Apply the function to the paths2 DataFrame
df['embeddings'] = df['simplified_path'].apply(lambda x: get_embeddings(x, articles_df))


/var/folders/sj/219f9qbn3y15yynshcyrrp3c0000gn/T/ipykernel_95506/2178106538.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['embeddings'] = df['simplified_path'].apply(lambda x: get_embeddings(x, articles_df))


In [59]:
# the target article is the last element in the simplified path
df = df.copy()
# Extract the target article and target embeddings
df['target_article'] = df['simplified_path'].apply(lambda x: x[-1] if isinstance(x, list) and x else None)
df['target_embeddings'] = df['embeddings'].apply(lambda x: x[-1] if isinstance(x, list) and x else None)

# Remove the last element from simplified_path and embeddings
df['simplified_path'] = df['simplified_path'].apply(lambda x: x[:-1] if isinstance(x, list) and x else x)
df['embeddings'] = df['embeddings'].apply(lambda x: x[:-1] if isinstance(x, list) and x else x)

In [61]:
# get the shape of the first element of the embeddings column

df.head(1)

,simplified_path,embeddings,target_article,target_embeddings
0,"[Capitalism, England, Great_Britain, Europe, A...","[[-0.0196740645915269, 0.1178837269544601, -0....",None,"[0.0680575370788574, 0.0213697664439678, 0.020..."


In [40]:

# Fetch the target article embedding
#paths2['target_embedding'] = 
paths2['target_embedding'] = paths2['target_article'].apply(
    lambda x: articles_df.loc[articles_df['article'] == x, 'embeddings'].values[0] if x in articles_df['article'].values else None
)



/var/folders/sj/219f9qbn3y15yynshcyrrp3c0000gn/T/ipykernel_95506/543675424.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  paths2['target_embedding'] = paths2['target_article'].apply(


In [43]:
paths2['target_embedding'].isnull().sum()

np.int64(28)